In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def tidy_split(df, column, sep=',', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df




In [2]:
# Read titles information
titles_info = pd.read_csv('/Users/Apple/Downloads/title.basics.tsv', delimiter = '\t', low_memory=False)
titles = titles_info.drop(columns = ['originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'], axis = 1)

#Read names information
names_info = pd.read_csv('/Users/Apple/Downloads/name.basics.tsv', delimiter = '\t', low_memory=False)
names_info = names_info.drop(columns = ['birthYear', 'deathYear', 'primaryProfession' ])

# Read ratings information
ratings_info = pd.read_csv('/Users/Apple/Downloads/title.ratings.tsv.gz', delimiter='\t', low_memory = False)


# Directors information
# Drop 'writer' column and merge with ratings info.
director_info = pd.read_csv('/Users/Apple/Downloads/title.crew.tsv', delimiter ='\t')
directors_info = director_info.drop(columns = ['writers'], axis =1)
directors_info = directors_info.merge(ratings_info, on = 'tconst', how = 'outer')

# Further clean by dropping all rows where 'director' information is missing and numVotes > 100,000
directors_info = directors_info.dropna(subset = ['directors'])
directors_info = directors_info[directors_info['numVotes']>100000]

# Retrieve only those titles that are movies and obtain their unique constants
titles = titles[titles['titleType']=='movie']
movie_titles = titles['tconst']

# Retrieve the director information only for titles that are movies
directors_info = directors_info[directors_info['tconst'].isin(movie_titles)]
directors_info.to_csv('')

# Duplicate rating information for titles with multiple directors
directors_info = tidy_split(directors_info, 'directors', ',')


done


In [69]:
def weighted_rating(df):
    """
    Compute the score for a given director as:
    sum over all movies (number of votes * average rating)/ # of movies made by the director

    Params
    -------
    df : Groupby object that contains the group information for the current director group
    numVotes : df param that contains the number of votes for all the movies
    averageRating : df param that contains average rating for all the movies

    Returns
    -------
    Float32
    Returns the score of the director for the current gro
    """

    a = (df.numVotes* df.averageRating)
    return a.sum()/len(df.directors)

def career_trajectory(df):
    p = []
    a = len(df.directors)
    b = df.averageRating
    xx = range(a)
    yy = b
    return xx,y
    

# Group by director name and compute score for each director
scores = []
names = []
p = []
x = []
y = []
groups = directors_info.groupby('directors')
for name, group in groups:
    if len(group.directors)> 4:
        names.append(name)
        scores.append(weighted_rating(group))
print 





[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [58]:
directors_scores = dict(zip(names,scores))

dnames = pd.DataFrame(directors_scores.items(), columns = ['names', 'scores'])
col1 = pd.Series(x_movieNumber)
col2 = pd.Series(y_rating) 
dnames['x_movieNumber']= col1.values
dnames['y_rating']=col2.values


dnames = dnames.sort_values(by = ['scores'], ascending = False).head(10)
n = dnames['names']
Director_names = names_info[names_info['nconst'].isin(n)]
y_scores = dnames['scores'].tolist()
y_scores = [int(i) for i in y_scores]
y_scores = [i/float(1000000) for i in y_scores]
x_names = Director_names['primaryName'].tolist()
print y_scores, x_names

x_movies = dnames['x_movieNumber']
y_rates = dnames['y_rating']
print x_movies.tolist()
print "\n"
print y_rates.tolist()









[9.180692, 6.418246, 5.646128, 5.60674, 4.895896, 4.815575, 4.416884, 4.282824, 4.215522, 4.215522] ['James Cameron', 'Martin Scorsese', 'Quentin Tarantino', 'Francis Ford Coppola', 'David Fincher', 'Peter Jackson', 'Christopher Nolan', 'Lee Unkrich', 'Lilly Wachowski', 'Lana Wachowski']
[5, 7, 6, 13, 7, 5, 9, 5, 8, 7]


[7.340000000000001, 7.300000000000002, 7.033333333333334, 7.423076923076922, 7.114285714285715, 7.3, 7.122222222222222, 7.1, 6.6875, 6.542857142857144]


In [ ]:
fig, ax = plt.subplots()

rects = ax.bar(np.arange(10), y_scores, tick_label=x_names)
ax.set_title('Top 10 directors of all time', fontsize=20)
ax.set_xlabel('Director', fontsize=18)
ax.set_ylabel('Score', fontsize=18)
ax.set_ylim(0, 10)

plt.setp(ax.get_xticklabels(), rotation=15, horizontalalignment='right')
plt.show()


